# Análise de métricas do TikTok - Caso VW Jetta

### Prof. Matheus C. Pestana

## Bibliotecas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

## Importando os dados e vendo as primeiras linhas

In [ ]:
jetta = pd.read_csv('../bases/jetta_tiktok.csv')
jetta.head()

## Análise exploratória

Qual o tamanho da base?

In [ ]:
n_linhas, n_colunas = jetta.shape
print(f"""O dataset possui:
- {n_linhas} linhas
- {n_colunas} colunas""")

### Informações sobre as variáveis

In [ ]:
jetta.info()

## Variáveis numéricas

### Média, desvio padrão, mínimo, máximo e quartis das variáveis numéricas

In [ ]:
jetta.describe()

### Plotando essas variáveis

In [ ]:
# um gráfico
plt.figure(figsize=(15, 10))
plt.subplot(321)
sns.histplot(data=jetta, x='likes', bins=20, color='green')
# outro gráfico
plt.subplot(322)
sns.histplot(data=jetta, x='comments', bins=20, color='red')
# outro gráfico
plt.subplot(323)
sns.histplot(data=jetta, x='shares', bins=20, color='orange')
# mais um
plt.subplot(324)
sns.histplot(data=jetta, x='views', bins=20, color='blue')
# mais outro
plt.subplot(313)
sns.histplot(data=jetta, x='duration', bins=20, color='purple')
plt.show()

### Será que quanto mais views, mais likes? Vamos testar!

In [ ]:
plt.figure(figsize=(15, 10))
sns.jointplot(data=jetta, x='views', y='likes',
              line_kws={'color': 'red'},
              scatter_kws={'color': 'green'},
              kind='reg')
plt.show()

O que percebemos aqui é que os valores se concentram muito no início, ou seja, com menos de 5000000.
Vamos limitar nossa visualização para posts com até 550000 de likes e 3000000 de views.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax.set_xlim(0, 3000000)
ax.set_ylim(0, 550000)
sns.regplot(data=jetta, x='views', y='likes',
              line_kws={'color': 'red'},
              scatter_kws={'color': 'green'},
              ax=ax)
plt.show()

### Será que quanto mais views, mais comentários? Vamos testar!

In [ ]:
plt.figure(figsize=(15, 10))
sns.jointplot(data=jetta, x='views', y='comments',
              line_kws={'color': 'red'},
              scatter_kws={'color': 'green'},
              kind='reg')
plt.show()

In [ ]:
sns.pairplot(data=jetta, vars=['likes', 'comments', 'shares', 'views', 'duration'], kind='reg')

### Estatísticas por autor

In [ ]:
# Calculando média, mínimo e máximo de cada uma das variáveis numéricas. O count é para contar o número de vídeos postados por cada autor, podendo ser usado em qualquer variável (pois contará o número de linhas)
jetta.groupby('user').agg({'likes': ['count','mean', 'min', 'max'],
                           'comments': ['mean', 'min', 'max'],
                           'shares': ['mean', 'min', 'max'],
                           'views': ['mean', 'min', 'max'],
                           'duration': ['mean', 'min', 'max']}).round(2).sort_values(by=('likes', 'mean'), ascending=False).reset_index()

Lembrando que a coluna likes count é, na verdade, o número de vídeos.

# Pergunta: Se eu fosse a Volkswagen e fosse escolher 3 influencers para fazer uma campanha, quais eu escolheria? Justifique sua resposta baseando-se nas métricas.

## Tempo

Como será que os vídeos sobre o Jetta estão distribuídos ao longo do tempo?

In [ ]:
jetta_tempo = jetta.copy()
# Transformando a coluna date em formato de data do Python
jetta_tempo['date'] = pd.to_datetime(jetta_tempo['date'])
# Criando uma coluna com o mês e o ano
jetta_tempo['month_year'] = jetta_tempo['date'].dt.to_period('M')
# Transformando a coluna em string
jetta_tempo['month_year'] = jetta_tempo['month_year'].astype(str)
# Contando o número de vídeos por mês
jetta_tempo = jetta_tempo.groupby('month_year')['likes'].count().reset_index().rename(columns={'likes': 'count'})
# Plotando o gráfico
plt.figure(figsize=(20, 10))
sns.lineplot(data=jetta_tempo, x='month_year', y='count')

### Agora vamos analisar como os likes estão distribuídos ao longo do tempo e juntar os dois gráficos

In [ ]:
# Fazer o mesmo para os likes
jetta_tempo_likes = jetta.copy()
jetta_tempo_likes['date'] = pd.to_datetime(jetta_tempo_likes['date'])
jetta_tempo_likes['month_year'] = jetta_tempo_likes['date'].dt.to_period('M')
jetta_tempo_likes['month_year'] = jetta_tempo_likes['month_year'].astype(str)
jetta_tempo_likes = jetta_tempo_likes.groupby('month_year')['likes'].sum().reset_index()
jetta_tempo_likes

In [ ]:
plt.subplot(211)
sns.lineplot(data=jetta_tempo, x='month_year', y='count')
plt.subplot(212)
sns.lineplot(data=jetta_tempo_likes, x='month_year', y='likes')
plt.show()

## Quais as hashtags mais usadas?

O Tiktok chama as hashtags de Challenges

In [ ]:
jetta_challenges = jetta.copy()
jetta_challenges['challenges'] = jetta_challenges['challenges'].str.split(', ')
# Conta todas os itens da lista com Counter
pd.DataFrame(Counter(jetta_challenges['challenges'].sum()).most_common(30))

### Nuvem de palavra com as hashtags

In [ ]:
wordcloud = WordCloud(width=800, height=400, max_font_size=150, max_words=100, background_color='white').generate(' '.join(jetta_challenges['challenges'].sum()))
plt.figure(figsize=(12, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()